In [102]:
import json
import pandas as pd
from pandas.io.json import json_normalize #package for flattening json in pandas df
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
with open('C:\\Users\\chq-shal\\Downloads\\projects_python\\project_workboard\\small3.json',encoding='utf-8') as d:
    f=json.load(d)
data=json_normalize(f)

In [103]:
begin=pd.to_datetime(data.beginTime, utc=True)
end=pd.to_datetime(data.endTime, utc=True)
compliant=end-begin<np.timedelta64(2,'h') # true is compliant, false is not compliant

In [104]:
payload=data['payload'].apply(lambda row:pd.Series(json.loads(row))) # convert payload dictionaries to columns
visibility=data['visibility'].apply(lambda row:pd.Series(json.loads(row))) # convert visibility dictionaries to columns

Interesting features to be added: pcsAct, wgtAct

In [105]:
# list(payload)
# payload.count()
# payload.typeOfGoods.value_counts()
payload[['pcsAct', 'wgtAct', 'wgtDeclAct']].count() # wgtDeclAct is the same as wgtAct

pcsAct        9564
wgtAct        7286
wgtDeclAct    7951
dtype: int64

Rows do not have any milestone codes

In [46]:
# list(visibility)
# visibility.count()
# milestone=visibility[['dsn','els','frd','phd','sir','cdp','cpu','dra','drr','sli','osd','puo','lkp','smp','ecs','evr','ntc','cln','afs','sfs','ecc','cek','hot','aue','frf','ega','rud','ddn']]
# milestone.count()
# len(milestone)-len(milestone.dropna(how='all')) 

In [106]:
df=pd.concat([begin,payload,visibility],axis=1)
df=df.dropna(axis=1) # drop columns that have na values in them
df=df.drop(['events','firstAware','etmsBranch','workBranch'],axis=1) # drop dict events, firstAware; drop dup etms, work
df=df.loc[:,~df.columns.duplicated()] # drop dup columns

Create additional features from dateCreate and beginTime, run this before the next code block

In [107]:
df['dateDT']=pd.to_datetime(df.dateCreate, utc=True)
df['diffS']=(df.dateDT-df.beginTime).dt.total_seconds()
df['beginY']=df.beginTime.dt.dayofyear
df['beginW']=df.beginTime.dt.dayofweek
df['beginH']=df.beginTime.dt.hour
df['dateY']=df.dateDT.dt.dayofyear
df['dateW']=df.dateDT.dt.dayofweek
df['dateH']=df.dateDT.dt.hour

In [108]:
too_sim=0.0003*len(df) # very similar values in a column
too_dif=0.5*len(df) # very different values in a column
for col in df.columns: # remove columns that have values that are too similar or too different
    if df[col].nunique()<too_sim or df[col].nunique()>too_dif:
        df.drop(col,inplace=True,axis=1)

Add interesting features back: pcsAct, wgtAct back. Drop rows where there is no wgtAct back.

In [111]:
df[['pcsAct', 'wgtAct']]=payload[['pcsAct', 'wgtAct']]
df=df.dropna()

In [119]:
compliance=compliant[pd.notnull(payload['wgtAct'])]
X_train,X_test,y_train,y_test=train_test_split(pd.get_dummies(df),compliance,random_state=0)
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

C:\Users\chq-shal\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\chq-shal\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.
C:\Users\chq-shal\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  """


In [122]:
from sklearn.ensemble import GradientBoostingClassifier
# clf=GradientBoostingClassifier(max_depth=2,random_state=0)
# grid_values = {'learning_rate': [0.001, 0.01, 0.05, 0.1, 1, 10]}
# grid_clf_auc = GridSearchCV(clf, param_grid = grid_values, scoring = 'roc_auc')
# grid_clf_auc.fit(X_train_scaled, y_train)
# y_decision_fn_scores_auc = grid_clf_auc.decision_function(X_test_scaled)
# print('Test set AUC: ', roc_auc_score(y_test, y_decision_fn_scores_auc))
# print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
# print('Grid best score (AUC): ', grid_clf_auc.best_score_)
clf=GradientBoostingClassifier(learning_rate=1,max_depth=2,random_state=0) # build bradient boost model with learning rate=1
gb_clf=clf.fit(X_train_scaled, y_train)
gb_y_score=gb_clf.decision_function(X_test_scaled)
gb_fpr, gb_tpr, _ = roc_curve(y_test, gb_y_score)
gb_roc_auc = auc(gb_fpr, gb_tpr)
gb_roc_auc

0.8999751737835153

In [123]:
feature=gb_clf.feature_importances_
pd.Series(data=feature,index=list(X_train)).nlargest(5)

mode              0.289424
branch_HKG        0.100853
diffS             0.083802
beginY            0.057749
portOrigin_SFO    0.051174
dtype: float64

In [124]:
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier()
# grid_values = {'n_estimators': [200, 700],'max_features': ['auto', 'sqrt', 'log2']}
# grid_clf_auc = GridSearchCV(clf, param_grid = grid_values, scoring = 'roc_auc')
# grid_clf_auc.fit(X_train_scaled, y_train)
# y_decision_fn_scores_auc = grid_clf_auc.predict_proba(X_test_scaled)[:,1]
# print('Test set AUC: ', roc_auc_score(y_test, y_decision_fn_scores_auc))
# print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
# print('Grid best score (AUC): ', grid_clf_auc.best_score_)
clf=RandomForestClassifier(max_features='log2', n_estimators=700)
rf_clf=clf.fit(X_train_scaled, y_train)
rf_y_score=rf_clf.predict_proba(X_test_scaled)[:,1]
rf_fpr, rf_tpr, _ = roc_curve(y_test, rf_y_score)
rf_roc_auc = auc(rf_fpr, rf_tpr)
rf_roc_auc

0.9491018751095274

In [125]:
feature=rf_clf.feature_importances_
pd.Series(data=feature,index=list(X_train)).nlargest(5) # mode is similar to shipment type

wgtAct    0.103063
diffS     0.094701
beginH    0.077398
dateH     0.071397
beginY    0.049330
dtype: float64

In [126]:
from sklearn.neighbors import KNeighborsClassifier
# clf = KNeighborsClassifier(n_jobs=-1)
# grid_values = {'n_neighbors': list(range(1,11))}
# grid_clf_auc = GridSearchCV(clf, param_grid = grid_values, scoring = 'roc_auc')
# grid_clf_auc.fit(X_train_scaled, y_train)
# y_decision_fn_scores_auc = grid_clf_auc.predict_proba(X_test_scaled)[:,1]
# print('Test set AUC: ', roc_auc_score(y_test, y_decision_fn_scores_auc))
# print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
# print('Grid best score (AUC): ', grid_clf_auc.best_score_)
clf=KNeighborsClassifier(n_neighbors=7,n_jobs=-1) # build knn model with neighbors=7
knn_clf=clf.fit(X_train_scaled, y_train)
knn_y_score=knn_clf.predict_proba(X_test_scaled)[:,1]
knn_fpr, knn_tpr, _ = roc_curve(y_test, knn_y_score)
knn_roc_auc = auc(knn_fpr, knn_tpr)
knn_roc_auc

0.8938817980022198

In [127]:
from sklearn.linear_model import LogisticRegression
# clf = LogisticRegression(n_jobs=-1)
# grid_values = {'C': [1,10,100,1000]}
# grid_clf_auc = GridSearchCV(clf, param_grid = grid_values, scoring = 'roc_auc')
# grid_clf_auc.fit(X_train_scaled, y_train)
# y_decision_fn_scores_auc = grid_clf_auc.decision_function(X_test_scaled)
# print('Test set AUC: ', roc_auc_score(y_test, y_decision_fn_scores_auc))
# print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
# print('Grid best score (AUC): ', grid_clf_auc.best_score_)
clf=LogisticRegression(C=1,n_jobs=-1) # build logistic regression model with C=1
lr_clf=clf.fit(X_train_scaled, y_train)
lr_y_score=lr_clf.predict_proba(X_test_scaled)[:,1]
lr_fpr, lr_tpr, _ = roc_curve(y_test, lr_y_score)
lr_roc_auc = auc(lr_fpr, lr_tpr)
lr_roc_auc

C:\Users\chq-shal\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\chq-shal\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


0.8540320696302356

In [128]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
dt_clf=clf.fit(X_train_scaled, y_train)
dt_y_score=dt_clf.predict_proba(X_test_scaled)[:,1]
dt_fpr, dt_tpr, _ = roc_curve(y_test, dt_y_score)
dt_roc_auc = auc(dt_fpr, dt_tpr)
dt_roc_auc

0.8286932647935041